In [ ]:
%load_ext autoreload
%autoreload 2
import logging
logging.basicConfig(level=logging.INFO)
import matplotlib.pyplot as plt

# from folium import LayerControl
from eo_tools_dev.util import show_insar_coh, show_insar_phi, show_sar_int
import geopandas as gpd
from eodag import EODataAccessGateway
from pathlib import Path

repo_dir = Path.cwd().parent
data_dir = repo_dir / "data"

# credentials need to be stored in the following file (see EODAG docs)
confpath = data_dir / "eodag.yml"
dag = EODataAccessGateway(user_conf_file_path=confpath)
# make sure cop_dataspace will be used 
dag.set_preferred_provider("cop_dataspace")
logging.basicConfig(level=logging.INFO)

# Set up parameters and output dir

In [ ]:

ids = [
 'S1A_IW_SLC__1SDV_20200603T063027_20200603T063054_032849_03CE08_23B4', 
 'S1A_IW_SLC__1SDV_20240606T063047_20240606T063114_054199_069770_E082'
]
primary_path = data_dir /f"{ids[0]}.zip"
secondary_path = data_dir /f"{ids[1]}.zip"
output_dir= data_dir / "res/test-full-processor"

In [ ]:
print(primary_path)

# Download S-1 products

In [ ]:
# load a geometry
aoi_file = data_dir / "grant_trees_aoi.shp"
gdf = gpd.read_file(aoi_file).to_crs("EPSG:4326")
shp = gdf[gdf["id"]==1].geometry.iloc[0]
search_criteria = {
    "productType": "S1_SAR_SLC",
    "start": "2020-06-01",
    "end": "2020-06-15",
    "geom": shp
}
results = dag.search_all(**search_criteria)
to_dl = [it for it in results if it.properties["id"] in ids]
print(f"{len(to_dl)} products to download")
dag.download_all(to_dl, output_dir=data_dir, extract=False)

In [ ]:
search_criteria = {
    "productType": "S1_SAR_SLC",
    "start": "2024-06-01",
    "end": "2024-06-15",
    "geom": shp
}
results = dag.search_all(**search_criteria)
to_dl = [it for it in results if it.properties["id"] in ids]
print(f"{len(to_dl)} products to download")
dag.download_all(to_dl, output_dir=data_dir, extract=False)

# Pre-process InSAR pair

In [ ]:
from eo_tools.S1.process import process_insar

out_dir = process_insar(
    prm_path='/app/data/S1A_IW_SLC__1SDV_20200603T063027_20200603T063054_032849_03CE08_23B4.SAFE.zip',
    sec_path='/app/data/S1A_IW_SLC__1SDV_20240606T063047_20240606T063114_054199_069770_E082.SAFE.zip',
    output_dir=output_dir,
    aoi_name=None,
    shp=shp,
    pol="vh",
    subswaths=["IW1", "IW2", "IW3"],
    write_coherence=True,
    write_interferogram=True,
    write_primary_amplitude=True,
    write_secondary_amplitude=False,
    apply_fast_esd=True,
    dem_upsampling=1.8,
    dem_force_download=False,
    dem_buffer_arc_sec=40,
    boxcar_coherence=[3, 3],
    filter_ifg=True,
    multilook=[1, 4],
    warp_kernel="bicubic",
    clip_to_shape=True,
)

In [ ]:
show_insar_coh(f"{out_dir}/coh_vh.tif")

In [ ]:
show_insar_phi(f"{out_dir}/phi_vh.tif")

In [ ]:
show_sar_int(f"{out_dir}/amp_prm_vh.tif")